# TensorFlow Recommenders: Quickstart

In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

# Import TensorFlow Recommender System (TFRS)

In [ ]:
!pip install -q tensorflow==2.3.0
!pip install -q tensorflow-recommenders==0.1.3
!pip install -q tensorflow-datasets==3.2.1

# Annoy
https://github.com/spotify/annoy

In [ ]:
!pip install -q annoy==1.16.3

In [1]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Load Ratings

In [2]:
ratings = tfds.load('movie_lens/100k-ratings', split="train")
print(ratings)

<PrefetchDataset shapes: {bucketized_user_age: (), movie_genres: (None,), movie_id: (), movie_title: (), raw_user_age: (), timestamp: (), user_gender: (), user_id: (), user_occupation_label: (), user_occupation_text: (), user_rating: (), user_zip_code: ()}, types: {bucketized_user_age: tf.float32, movie_genres: tf.int64, movie_id: tf.string, movie_title: tf.string, raw_user_age: tf.float32, timestamp: tf.int64, user_gender: tf.bool, user_id: tf.string, user_occupation_label: tf.int64, user_occupation_text: tf.string, user_rating: tf.float32, user_zip_code: tf.string}>


# Load Movies

In [3]:
movies = tfds.load('movie_lens/100k-movies', split="train")
print(movies)

<PrefetchDataset shapes: {movie_genres: (None,), movie_id: (), movie_title: ()}, types: {movie_genres: tf.int64, movie_id: tf.string, movie_title: tf.string}>


# Select Basic Features

In [4]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
print(ratings)

<MapDataset shapes: {movie_title: (), user_id: ()}, types: {movie_title: tf.string, user_id: tf.string}>


In [5]:
movies = movies.map(lambda x: x["movie_title"])
print(movies)

<MapDataset shapes: (), types: tf.string>


In [6]:
#print(type(ratings))
#print(type(movies))

# Create Vocabularies
Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [7]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

In [8]:
#print(user_ids_vocabulary.get_vocabulary())

# Create the Model

We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

In [13]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

# Define User and Movie Models
Define the two models.

In [14]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 128)
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 128)
])

# Define the Retrieval Task

In [ ]:
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

# Train the Retrieval Model

In [ ]:
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train.
model.fit(ratings.batch(4096), epochs=100)

Epoch 1/100
25/25 [==============================] - 6s 244ms/step - factorized_top_k: 0.0233 - factorized_top_k/top_1_categorical_accuracy: 3.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 8.0000e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0030 - factorized_top_k/top_50_categorical_accuracy: 0.0338 - factorized_top_k/top_100_categorical_accuracy: 0.0786 - loss: 35764.4455 - regularization_loss: 0.0000e+00 - total_loss: 35764.4455
Epoch 2/100
25/25 [==============================] - 6s 245ms/step - factorized_top_k: 0.0648 - factorized_top_k/top_1_categorical_accuracy: 1.6000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0036 - factorized_top_k/top_10_categorical_accuracy: 0.0113 - factorized_top_k/top_50_categorical_accuracy: 0.1005 - factorized_top_k/top_100_categorical_accuracy: 0.2085 - loss: 31245.6944 - regularization_loss: 0.0000e+00 - total_loss: 31245.6944
Epoch 3/100
25/25 [==============================] - 6s 245ms/step - factorized_top_k: 0.0919 - 

25/25 [==============================] - 6s 244ms/step - factorized_top_k: 0.1772 - factorized_top_k/top_1_categorical_accuracy: 0.0010 - factorized_top_k/top_5_categorical_accuracy: 0.0146 - factorized_top_k/top_10_categorical_accuracy: 0.0385 - factorized_top_k/top_50_categorical_accuracy: 0.3031 - factorized_top_k/top_100_categorical_accuracy: 0.5289 - loss: 27820.6092 - regularization_loss: 0.0000e+00 - total_loss: 27820.6092
Epoch 38/100
25/25 [==============================] - 6s 245ms/step - factorized_top_k: 0.1776 - factorized_top_k/top_1_categorical_accuracy: 9.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0147 - factorized_top_k/top_10_categorical_accuracy: 0.0387 - factorized_top_k/top_50_categorical_accuracy: 0.3041 - factorized_top_k/top_100_categorical_accuracy: 0.5298 - loss: 27800.8864 - regularization_loss: 0.0000e+00 - total_loss: 27800.8864
Epoch 39/100
25/25 [==============================] - 6s 246ms/step - factorized_top_k: 0.1781 - factorized_top_k/t

# Make Predictions
Use brute-force search to set up retrieval using the trained representations.

In [12]:
index = tfrs.layers.ann.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Top 3 recommendations for user 42: [b'Rent-a-Kid (1995)' b'House Arrest (1996)' b'Clean Slate (1994)']
